# Ancillary Datasets

In order to use external/ancillary datasets as feature layers in the crop ML model, we need to resample the datasets to match Sentinel2.  The code here tests the workflow for resampling.  

Also testing creating a slope layer.

`xr_reproject` is a new function and needs to be installed manually as per below: 

     !pip install -U --no-deps --extra-index-url="https://packages.dea.ga.gov.au" odc-algo

### Try reprojecting CHIRPS

Issue with `.geobox` expecting lat/lon as coords rather than xy, will be fixed in `ODC1.8.1`

In [ ]:
!pip install -U --no-deps --extra-index-url="https://packages.dea.ga.gov.au" odc-algo

In [ ]:
from odc.algo import xr_reproject
from datacube.utils.geometry import assign_crs
import xarray as xr
import datacube
import sys
sys.path.append('../Scripts')
from deafrica_datahandling import load_ard

In [ ]:
dc = datacube.Datacube(app='Vegetation_phenology')

# Define area of interest
lat = 22.817
lon = 28.518
lon_buffer = 1.0 #0.0175
lat_buffer = 1.0 #0.004

# Combine central lat,lon with buffer to get area of interest
lat_range = (lat-lat_buffer, lat+lat_buffer)
lon_range = (lon-lon_buffer, lon+lon_buffer)

# Set the range of dates for the analysis
years_range = ('2018')

In [ ]:
# Create a reusable query
query = {
    'y': lat_range,
    'x': lon_range,
    'time': years_range,
    'measurements': ['red', 'green', 'blue', 'nir'],
    'resolution': (-30,30),
    'output_crs': 'epsg:6933'
}

# Load available data from Landsat 8
ds = dc.load(product='ga_ls8c_gm_2_annual',
              **query,
              )

print(ds)

In [ ]:
chirps = xr.open_rasterio('data/CHIRPS/CHPclim.01.tif')

In [ ]:
chrips_30m = xr_reproject(chirps,
                 ds.geobox,
                 "mode",
                    )

## Calculate Slope on SRTM

    !pip install richdem

In [ ]:
import os
import datacube
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import richdem as rd
import pyproj
import xarray as xr


In [ ]:
lat, lon = -3.064, 37.359
buffer = 0.1

query = {
    'x': (lon-buffer, lon+buffer),
    'y': (lat+buffer, lat-buffer),
    'output_crs': 'EPSG:6933',
    'resolution': (-30, 30)
}

dc = datacube.Datacube(app='slope')

#load data
da = dc.load(product ='srtm', **query)
da = da.elevation

In [ ]:
def xr_terrain(da, attribute=None):
    """
    Using the richdem package, calculates terrain attributes
    on a DEM stored in memory as an xarray.DataArray 
    
    Params
    -------
    da : xr.DataArray
    attribute : str
        One of the terrain attributes that richdem.TerrainAttribute()
        has implemented. e.g. 'slope_riserun', 'slope_percentage', 'aspect'.
        See all option here:  
        https://richdem.readthedocs.io/en/latest/python_api.html#richdem.TerrainAttribute
        
    """
    #remove time if its there
    da = da.squeeze()
    #convert to richdem array
    rda = rd.rdarray(da.data, no_data=da.attrs['nodata'])
    #add projection and geotransform
    rda.projection=pyproj.crs.CRS(da.attrs['crs']).to_wkt()
    rda.geotransform = da.geobox.affine.to_gdal()
    #calulate attribute
    attrs = rd.TerrainAttribute(rda, attrib=attribute)

    #return as xarray DataArray
    return xr.DataArray(attrs,
                        attrs=da.attrs,
                        coords={'x':da.x, 'y':da.y},
                        dims=['y', 'x'])

In [ ]:
%%time
slope = xr_terrain(da, 'slope_riserun')

In [ ]:
slope.plot(cmap='magma', figsize=(8,8), robust=True);